# Advent of Code 2022 Solutions

In [1]:
import pandas as pd
from aoc import get_input_data

## Day 1

In [2]:
data = get_input_data(1)

s = pd.Series([sum(map(int, chunk.split('\n'))) for chunk in data.split('\n\n')])

# Part 1
print(s.max())

# Part 2
print(s.nlargest(3).sum())

70698
206643


## Day 2

In [3]:
data = """A Y
B X
C Z"""

data

'A Y\nB X\nC Z'

In [4]:
data = get_input_data(2)

### Part 1

First Column
> A for Rock, B for Paper, and C for Scissors.

Second column
> X for Rock, Y for Paper, and Z for Scissors

Scoring
> (1 for Rock, 2 for Paper, and 3 for Scissors)<br>
> (0 if you lost, 3 if the round was a draw, and 6 if you won)

In [5]:
d = {
    'A': {
        'X': (1, 3),
        'Y': (2, 6),
        'Z': (3, 0)
    },
    'B': {
        'X': (1, 0),
        'Y': (2, 3),
        'Z': (3, 6)
    },
    'C': {
        'X': (1, 6),
        'Y': (2, 0),
        'Z': (3, 3)
    },
}

score = 0

for x in data.split('\n'):
    p1, p2 = x.split()
    points, outcome = d[p1][p2]
    score += points + outcome

score

12586

### Part 2

> X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win

In [6]:
d_points = {
    'A': 1, 'B': 2, 'C': 3,
    'X': 1, 'Y': 2, 'Z': 3
}
d_win_lose = {'A': ('Y', 'Z'), 'B': ('Z', 'X'), 'C': ('X', 'Y')}


score = 0

for x in data.split('\n'):
    p1, p2 = x.split()
    
    win, lose = d_win_lose[p1]

    # Lose
    if p2 == 'X':
        outcome = 0
        points = d_points[lose]
    
    # Draw
    elif p2 == 'Y':
        outcome = 3
        points = d_points[p1]
        
    # Win
    else:
        outcome = 6
        points = d_points[win]
    score += outcome + points

score

13193

---
## Day 3
### Part 1

In [7]:
data = """vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw"""

data = data.split('\n')
data

['vJrwpWtwJgWrhcsFMMfFFhFp',
 'jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL',
 'PmmdzqPrVvPwwTWBwg',
 'wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn',
 'ttgJtRGJQctTZtZT',
 'CrZsJsPPZsGzwwsLwLmpwMDw']

In [8]:
data = get_input_data(3)
data = data.split('\n')

In [9]:
from string import ascii_lowercase, ascii_uppercase

# Build letter priority dict
priorities = {l: i for i, l in enumerate(ascii_lowercase, 1)}
priorities.update({l: i for i, l in enumerate(ascii_uppercase, 27)})
print(priorities)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'Z': 52}


In [10]:
def split_rucksack(rucksack):
    mid = len(rucksack) // 2
    return rucksack[:mid], rucksack[mid:]

def get_common_items(compartment1, compartment2):
    c1 = set(compartment1)
    c2 = set(compartment2)
    return c1 & c2

In [11]:
score = 0

for rucksack in data:
    compartments = split_rucksack(rucksack)
    common_items = get_common_items(*compartments)
    for item in common_items:
        score += priorities.get(item)
score

8515

### Part 2

In [12]:
def get_common_items_v2(*rucksacks):
    items = [set(rucksack) for rucksack in rucksacks]
    return set.intersection(*items)


In [13]:
score = 0
n_group = 3

for i in range(0, len(data), n_group):
    group_rucksacks = data[i: i + n_group]
    common_items = get_common_items_v2(*group_rucksacks)
    for item in common_items:
        score += priorities.get(item)
score

2434

## Day 4
### Part 1

In [14]:
data = '''2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8'''


In [15]:
def process_line(line):
    ranges = dict(enumerate(line.split(',')))
    for i, rng in ranges.items():
        start, end = map(int, rng.split('-'))
        rng = range(start, end+1)
        ranges[i] = rng
    return tuple(ranges.values())

def order_ranges(*ranges):
    return sorted(ranges, key=lambda x: x.start - x.stop)

def is_contained(outer, inner):
    if (outer.start <= inner.start) and (outer.stop >= inner.stop):
        return True
    return False


In [16]:
data = get_input_data(4)

In [17]:
%%time
count = 0

for line in data.split('\n'):
    rng1, rng2 = process_line(line)
    ordered = order_ranges(rng1, rng2)
    if is_contained(*ordered):
        count += 1
count

CPU times: user 7.98 ms, sys: 220 µs, total: 8.2 ms
Wall time: 8.43 ms


431

### Part 2

In [18]:
def overlaps(rng1, rng2):
    if set(rng1).intersection(rng2):
        return True
    if set(rng2).intersection(rng1):
        return True
    return False

In [19]:
%%time

count = 0

for line in data.split('\n'):
    rng1, rng2 = process_line(line)
    if overlaps(rng1, rng2):
        count += 1
count

CPU times: user 5.13 ms, sys: 155 µs, total: 5.29 ms
Wall time: 5.24 ms


823